# Exploratory Data Analysis (EDA) and data cleaning notebook #

**Importing libraries and modules**

In [321]:
# importing external libraries
from pathlib import Path
import os
import pandas as pd
import pickle
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config
set_config(transform_output = "pandas")

# Importing function to load data

# Making sure any changes are instantly added
%load_ext autoreload
%autoreload 2

from Modules.load_data import load_data
from Modules.preprocessing import missing_summary, merge_dfs, dollar_to_int, find_unique_values
from Modules.plotting import Plotter
from Modules.transforming import *



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Please unhash and run the cell below if you have not yet installed the dataset using the kaggle API

In [3]:
#load_data()

– Describe your data (e.g. dtypes, descriptive statistics)
– What is the distribution of the target variable?
– Do we face missing values / outliers?
– How do specific features correlate with the target variable?
– What features can we use for the specific prediction task?

– Describe your data (e.g. dtypes, descriptive statistics)
– What is the distribution of the target variable?
– Do we face missing values / outliers?
– How do specific features correlate with the target variable?
– What features can we use for the specific prediction task?

In [4]:
# Obtaining absolute path to data folder
data_folder = str(Path(os.getcwd()) / "data")

# Obtaining absolute paths to relevant datasets

cards_data = data_folder + "/cards_data.csv"
transaction_data = data_folder + "/transactions_data.csv"

**Pre-processing steps for transaction data**

 * `date` column to be decomposed into seperate month, data, time columns if there is correlation between time of day, day of week etc. and fraduluent transactions
 * `merchant_id` represents the business where transaction was made. Likely too many clients to use as categorical variable. Possible do mean-encoding.
 * `card_id`. May be possible to somehow represent if cardholder has been flagged for fradulent transaction before
 * `client_id` similar type of encoding as for merchant id.
 * `merchant_city` and `merchant_state` likely one-hot encoding.
 * `zip`, perhaps any predictability is covered in the other location variables. Check for correlation and then drop.
 * `mcc` represents the type of merchant. Possible one hot encoding or mean encoding.
 * `errors`. Over 98% missing. Look for correlations then maybe drop.
 * Any columns with $ values are object type. Remove $ sign and change to int.



In [5]:
# Merging cards and transactions df and saving to pickle
# Unhash if this is the first time running the code

# Reading datasets into pandas
# cards_data_df = pd.read_csv(cards_data)
# transaction_data_df = pd.read_csv(transaction_data)

# merge_dfs(transaction_data_df=transaction_data_df, cards_data_df=cards_data_df,data_folder=data_folder)

In [6]:
# Loading the data from pickle

merged_df = pd.read_pickle(data_folder + "/merged_data.pkl")

In [7]:
# Viewing head of merged_df
display(merged_df.head())

,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,...,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web,id,target
0,2010-01-01 00:01:00,1556,2972,$-77.00,Swipe Transaction,59935,Beulah,ND,58523.0,5499,...,07/2022,306,YES,2,$55,05/2008,2008,No,7475327,No
1,2010-01-01 00:02:00,561,4575,$14.57,Swipe Transaction,67570,Bettendorf,IA,52722.0,5311,...,12/2024,438,YES,1,$9100,09/2005,2015,No,7475328,No
2,2010-01-01 00:02:00,1129,102,$80.00,Swipe Transaction,27092,Vista,CA,92084.0,4829,...,05/2020,256,YES,1,$14802,01/2006,2008,No,7475329,No
3,2010-01-01 00:06:00,848,3915,$46.41,Swipe Transaction,13051,Harwood,MD,20776.0,5813,...,01/2020,120,YES,1,$19113,07/2009,2014,No,7475332,No
4,2010-01-01 00:07:00,1807,165,$4.81,Swipe Transaction,20519,Bronx,NY,10464.0,5942,...,03/2014,198,YES,1,$89,01/2008,2015,No,7475333,No


In [8]:


''' 
Potential encoding options
client_id: one_hot & mean imputation
'''

' \nPotential encoding options\nclient_id: one_hot & mean imputation\n'

In [9]:
# Running info to see column types
merged_df.info()

"""
As we can see a lot of columns that should be numerical are objects as they have dollar signs.
A lot of columns need to be made numerical
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date                   object 
 1   client_id              int64  
 2   card_id                int64  
 3   amount                 object 
 4   use_chip               object 
 5   merchant_id            int64  
 6   merchant_city          object 
 7   merchant_state         object 
 8   zip                    float64
 9   mcc                    int64  
 10  errors                 object 
 11  card_brand             object 
 12  card_type              object 
 13  card_number            int64  
 14  expires                object 
 15  cvv                    int64  
 16  has_chip               object 
 17  num_cards_issued       int64  
 18  credit_limit           object 
 19  acct_open_date         object 
 20  year_pin_last_changed  int64  
 21  card_on_dark_web       object 
 22  id                

'\nAs we can see a lot of columns that should be numerical are objects as they have dollar signs.\nA lot of columns need to be made numerical\n'

**Items to consider**

* `date` column into hour of the day, day of the week, month of year, year, time series 0 - T
* Huge data disparity, also large data set so computationally expensive perhaps only need x% of all non-fraudulent transactions included

In [10]:
# Running the dollar_to_int function
dollar_to_int(merged_df)

In [11]:
# Running .info() again
merged_df.info()

""" 
'amount' and 'credit limit' now changed to int types
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8914963 entries, 0 to 8914962
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   date                   object 
 1   client_id              int64  
 2   card_id                int64  
 3   amount                 int64  
 4   use_chip               object 
 5   merchant_id            int64  
 6   merchant_city          object 
 7   merchant_state         object 
 8   zip                    float64
 9   mcc                    int64  
 10  errors                 object 
 11  card_brand             object 
 12  card_type              object 
 13  card_number            int64  
 14  expires                object 
 15  cvv                    int64  
 16  has_chip               object 
 17  num_cards_issued       int64  
 18  credit_limit           int64  
 19  acct_open_date         object 
 20  year_pin_last_changed  int64  
 21  card_on_dark_web       object 
 22  id                

" \n'amount' and 'credit limit' now changed to int types\n"

In [12]:
# Running describe
merged_df.describe()

,client_id,card_id,amount,merchant_id,zip,mcc,card_number,cvv,num_cards_issued,credit_limit,year_pin_last_changed,id
count,8.914963e+06,8.914963e+06,8.914963e+06,8.914963e+06,7.807586e+06,8.914963e+06,8.914963e+06,8.914963e+06,8.914963e+06,8.914963e+06,8.914963e+06,8.914963e+06
mean,1.026637e+03,3.474887e+03,4.252761e+01,4.772566e+04,5.132855e+04,5.565097e+03,4.817349e+15,4.953292e+02,1.522064e+00,1.554959e+04,2.011340e+03,1.558473e+07
std,5.816755e+02,1.674427e+03,8.151282e+01,2.581623e+04,2.940518e+04,8.755078e+02,1.311465e+15,2.885735e+02,5.151711e-01,1.218199e+04,2.894518e+00,4.703991e+06
min,0.000000e+00,0.000000e+00,-5.000000e+02,1.000000e+00,1.001000e+03,1.711000e+03,3.001055e+14,0.000000e+00,1.000000e+00,0.000000e+00,2.002000e+03,7.475327e+06
25%,5.190000e+02,2.413000e+03,8.000000e+00,2.588700e+04,2.860100e+04,5.300000e+03,4.489873e+15,2.470000e+02,1.000000e+00,8.100000e+03,2.010000e+03,1.150786e+07
50%,1.070000e+03,3.584000e+03,2.800000e+01,4.592600e+04,4.771000e+04,5.499000e+03,5.112842e+15,4.990000e+02,2.000000e+00,1.345500e+04,2.011000e+03,1.557140e+07
75%,1.530000e+03,4.899000e+03,6.300000e+01,6.757000e+04,7.790100e+04,5.812000e+03,5.566696e+15,7.400000e+02,2.000000e+00,2.083900e+04,2.013000e+03,1.965387e+07
max,1.998000e+03,6.138000e+03,6.613000e+03,1.003420e+05,9.992800e+04,9.402000e+03,6.994218e+15,9.990000e+02,3.000000e+00,1.413910e+05,2.020000e+03,2.376187e+07


In [13]:
"""
A lot of missing values for `errors` column 
"""

# Running missing summary
missing_summary(merged_df)



,Missing Values,Percentage missing (%)
date,0,0.000000
client_id,0,0.000000
card_id,0,0.000000
amount,0,0.000000
use_chip,0,0.000000
merchant_id,0,0.000000
merchant_city,0,0.000000
merchant_state,1047865,11.754003
zip,1107377,12.421555
mcc,0,0.000000


In [14]:
'''
A missing value likely to mean there was no transaction error
'''


merged_df["errors"][merged_df["errors"].notna()]


161              Bad Expiration
180             Bad Card Number
262        Insufficient Balance
319        Insufficient Balance
320        Insufficient Balance
                   ...         
8914572    Insufficient Balance
8914610    Insufficient Balance
8914635    Insufficient Balance
8914782                 Bad PIN
8914898    Insufficient Balance
Name: errors, Length: 141767, dtype: object

### Plots of Data

In [15]:
# Creating instance of plotter class
plot_data = Plotter(df=merged_df)

In [16]:
plot_data.display_target(target="target")

,Response,Frequency,Percentage
0,No,8901631,99.85
1,Yes,13332,0.15


We see a huge data disparity in the target variable, henceforth we will need to use different performance metrics to accuracy. We also need to do ensure a good distribution of responses in the train and test set.

In [322]:
## Creating pipeline

# Define the fillna transformer
fillna_transformer = FunctionTransformer(lambda X: X.fillna(0))

# Define a function to copy the target column
def copy_target_column(X):
    X = X.copy()  # Ensure no modification to the original DataFrame
    X["target_copy"] = X["target"]
    return X

target_copy_transformer = FunctionTransformer(copy_target_column, validate=False)

reduction_pipeline = Pipeline(
    steps=[
        ( "Return_reduced_df", Target0_Reducer(percentage=0.01) ),
        (  "Make_target_binary", TargetBinary(type="df")   ),
        ("Fill_NA", fillna_transformer),  # Add the fillna step
        ("Add_target_copy", target_copy_transformer),
    ])

binary_pipeline = Pipeline(
    steps=[
        (  "Make_target_binary", TargetBinary(type="column")   )
    ])

time_series_pipeline1 = Pipeline(
    steps=[
    ( "Convert_to_dt", Date()  ),
    ( "ts_mapping", TimeSeriesMapper() )
])

time_series_pipeline2 = Pipeline(
    steps=[
    ( "ts_mapping", TimeSeriesMapper() )
])

one_hot_pipeline = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

target_encoder_pipeline = Pipeline([
    ("Target_encoder", CustomTargetEncoder(target="target"))
])

numerical_pipeline = Pipeline([
    ("dollar_to_int", DollarToInt()),
    ("min_max_scaler", MinMaxScaler())
])


In [323]:
reduced_df["has_chip"].str.lower().map({"yes":1,"no":0})

0         1
1         0
2         1
3         1
4         1
         ..
102343    1
102344    1
102345    1
102346    1
102347    1
Name: has_chip, Length: 102348, dtype: int64

In [324]:
all_columns = list(merged_df.columns)

preprocessor = ColumnTransformer([
    ("a", time_series_pipeline1, "date"),
    ("b", time_series_pipeline1, "acct_open_date"),
    ("c", binary_pipeline, "card_on_dark_web"),
    ("d", one_hot_pipeline, ["use_chip"]),
    ("e", one_hot_pipeline, ["card_brand"]),
    ("f", one_hot_pipeline, ["card_type"]),
    ("g", binary_pipeline, "has_chip"),
    ("h", target_encoder_pipeline, ["client_id", "target"]),
    ("i", target_encoder_pipeline, ["card_id", "target"]),
    ("j", target_encoder_pipeline, ["merchant_id", "target"]),
    ("k", target_encoder_pipeline, ["merchant_city", "target"]),
    ("l", target_encoder_pipeline, ["merchant_state", "target"]),
    ("m", target_encoder_pipeline, ["zip", "target"]),
    ("n", target_encoder_pipeline, ["mcc", "target"]),
    ("o", target_encoder_pipeline, ["errors", "target"]),
    ("p", target_encoder_pipeline, ["card_number", "target"]),
    ("q", target_encoder_pipeline, ["cvv", "target"]),
    ("r", numerical_pipeline, "amount"),
    ("s", numerical_pipeline, "credit_limit"),
    ("t", time_series_pipeline1, "expires"),
     
],
remainder="passthrough"
)

In [278]:
# Returning reduced df based on first transformation

reduced_df = reduction_pipeline.fit_transform(merged_df)

In [325]:
binary_pipeline.fit_transform(reduced_df["has_chip"])

,has_chip
0,1
1,0
2,1
3,1
4,1
...,...
102343,1
102344,1
102345,1
102346,1


In [326]:
transform_target = preprocessor.fit_transform(reduced_df)

/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:55: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()
/home/as3620/FODS_coursework/LoanPrediction/Modules/transforming.py:55: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return pd.to_datetime(X).to_frame()


In [329]:
transform_target["c__card_on_dark_web"].unique()

array([0])

In [ ]:
''' 
client_id and card_id can be tested with
'''

find_unique_values(reduced_df)



,columns,unique_no,unique_%
0,date,805025,89.10
1,client_id,1219,0.13
2,card_id,4067,0.45
3,amount,1887,0.21
4,use_chip,3,0.00
5,merchant_id,29387,3.25
6,merchant_city,8907,0.99
7,merchant_state,171,0.02
8,zip,17665,1.96
9,mcc,109,0.01


In [327]:
transform_target.corr()["remainder__target_copy"]

a__date                             0.007473
b__acct_open_date                   0.006825
c__card_on_dark_web                      NaN
d__use_chip_Chip Transaction       -0.086101
d__use_chip_Online Transaction      0.468127
d__use_chip_Swipe Transaction      -0.283739
e__card_brand_Amex                  0.006361
e__card_brand_Discover              0.022973
e__card_brand_Mastercard           -0.001813
e__card_brand_Visa                 -0.008997
f__card_type_Credit                 0.020557
f__card_type_Debit                 -0.042813
f__card_type_Debit (Prepaid)        0.043214
g__has_chip                         0.010951
h__client_id                        0.300779
i__card_id                          0.488684
j__merchant_id                      0.685686
k__merchant_city                    0.729342
l__merchant_state                   0.696351
m__zip                              0.686538
n__mcc                              0.540882
o__errors                           0.109788
p__card_nu